<a href="https://www.kaggle.com/adityamahimkar/generative-adversarial-network-implementation?scriptVersionId=86747268" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import os 
import cv2
import osmnx as ox
!pip install geemap
import geemap
import geopandas as gpd
from PIL import Image
from numpy import asarray
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from matplotlib import pyplot
import matplotlib.pyplot as plt
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import (Input, MaxPooling2D, GlobalAveragePooling2D, 
                          UpSampling2D,Conv2DTranspose,LeakyReLU,Dropout,
                          Activation,Dense,Flatten,Reshape,Conv2D)

In [ ]:
cities = pd.read_csv('../input/world-cities-dataset/cities - Population.csv')
cities.head()

In [ ]:
cities_sort = cities.sort_values(by='Area(Sq Km)')
cities_sort.head()

In [ ]:
cities.shape

In [ ]:
from PIL import Image
import os
import numpy as np
import io
import cv2
import osmnx as ox
import geopandas as gpd

def get_img_from_fig(fig, dpi=200):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img


def cut_tiles(img, city_dir, tile_size):
    # Create directories
    if not os.path.exists(city_dir):
        os.makedirs(city_dir)

    # Create mask for current image
#   img1 = Image.open('./foo.png')
#   img = np.asarray(img1)
    x, y, _ = img.shape

    # Cut tiles and save them
    for i in range(x//tile_size):
        for j in range(y//tile_size):
            img_tile = img[i*tile_size:(i+1)*tile_size,j*tile_size:(j+1)*tile_size]
            Image.fromarray(img_tile).save(f'{city_dir}/{i}_{j}.png')
    
    

TILE_SIZE = 255
for i in range(cities_sort.shape[0]):
    city = cities_sort.iloc[i,1] + ', ' + cities_sort.iloc[i,2]
    print(city)
    try:
        graph = ox.graph_from_place(city)
        print('graph collected')
        fig, ax = plt.subplots(figsize=(40,40))
        nodes, edges = ox.graph_to_gdfs(graph)
        area = ox.geocode_to_gdf(city)
        print('area, edges collected')
        
        area.plot(ax=ax, facecolor='black')
        edges.plot(ax=ax, linewidth=1, edgecolor='#BC8F8F')
        plot_img_np = get_img_from_fig(fig)
        cut_tiles(plot_img_np, city, TILE_SIZE)
        print('Successful', city)
    
    except Exception as e:
        print('Unable to fetch', city)
    

In [ ]:
cities_sort['Name'][1]

In [ ]:
fig, ax = plt.subplots(figsize=(40,40))
nodes, edges = ox.graph_to_gdfs(graph)
area = ox.geocode_to_gdf('Mumbai, India')

area.plot(ax=ax, facecolor='black')
edges.plot(ax=ax, linewidth=1, edgecolor='#BC8F8F')

fig.savefig('foo.png', dpi=300)
# img = cv2.imread('./foo.png', 1)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# plt.imshow(img)

In [ ]:
import io
def get_img_from_fig(fig, dpi=200):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img

In [ ]:
plot_img_np = get_img_from_fig(fig)

In [ ]:
plot_img_np.shape

In [ ]:
img = Image.open('./img_tiles/12_25.png')

In [ ]:
# dont run

for dirname, _, filenames in os.walk('./img_tiles'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

In [ ]:
len(filenames)

In [ ]:
dataset_orig = np.empty((len(filenames), 64, 64,3))

In [ ]:
ind=0
for i in filenames:
    image = Image.open(os.path.join(dirname, i))
    data = asarray(image)
    dataset_orig[ind]=data
    ind+=1

In [ ]:
width, height, channel = 64, 64, 3
np.random.shuffle(dataset_orig)
X=dataset_orig

In [ ]:
X[1]

In [ ]:
X = (X - 127.5) / 127.5

In [ ]:
X[1]

In [ ]:
X.shape

In [ ]:
def show_data(X, title=""):
    plt.figure(figsize=(11,11))
    
    i = 1
    for img in X:
        plt.subplot(10, 10, i)
        plt.imshow(img.reshape((height, width,channel)))
        plt.axis('off')
        i+=1
        if i>100: break

    plt.suptitle(title, fontsize = 25)
    plt.show()
    
show_data(X, title="Original Dataset")

In [ ]:
gen_optimizer = Adam(0.0001, 0.5)
disc_optimizer = Adam(0.0002, 0.5)
noise_dim = 100

In [ ]:
def buildGenerator():
    model = Sequential()

    model.add(Dense(1024, input_dim=noise_dim))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(Dense(8192, input_dim=noise_dim))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(Reshape((8, 8, 128)))
    
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, (2, 2), padding='same', 
                     kernel_initializer=RandomNormal(0, 0.02)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(0.2))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, (2, 2), padding='same', 
                     kernel_initializer=RandomNormal(0, 0.02)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(0.2))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(channel, (3, 3), padding='same', activation = "tanh", 
                     kernel_initializer=RandomNormal(0, 0.02)))
    
    return model

In [ ]:
generator = buildGenerator()
generator.summary()
# plot the model
plot_model(generator, to_file='generator_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
def buildDiscriminator():
    model = Sequential()
    
    model.add(Conv2D(64, (5, 5), strides=2, padding='same', 
                     kernel_initializer=RandomNormal(0, 0.02), 
                     input_shape=(width, height, channel)))
    model.add(LeakyReLU(0.2))


    model.add(Conv2D(64, (5, 5), strides=2, padding='same', 
                     kernel_initializer=RandomNormal(0, 0.02), 
                     input_shape=(width, height, channel)))
    model.add(LeakyReLU(0.2))
    
    model.add(Conv2D(128, (5, 5), strides=2,padding='same', 
                     kernel_initializer=RandomNormal(0, 0.02)))
    model.add(LeakyReLU(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    
    model.add(Dropout(0.5))

    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=disc_optimizer)
    return model

In [ ]:
discriminator = buildDiscriminator()
discriminator.summary()
# plot the model
plot_model(discriminator, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
noise = Input(shape=(noise_dim,))
fake_data = generator(noise)
discriminator.trainable = False
output = discriminator(fake_data)
gan = Model(noise, output)
gan.compile(loss='binary_crossentropy', optimizer=gen_optimizer)

In [ ]:
gan.summary()
# plot the model
plot_model(gan, to_file='gan_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
fixed_noise = np.random.normal(0, 1, size=(100, noise_dim))

In [ ]:
def show_generated_fabric(title, epoch):
    imgs = generator.predict(fixed_noise)
    imgs = 0.5 * imgs + 0.5
    plt.figure(figsize=(11,11))
    
    i = 1
    for img in imgs:
        plt.subplot(10, 10, i)
        plt.imshow(img.reshape((height,width,channel)))
        plt.axis('off')
        i+=1
    plt.suptitle(title, fontsize = 25)
    plt.savefig(str(epoch+1)+".png", transparent=True)
    plt.show()

In [ ]:
epochs = 500
batch_size = 128
steps_per_epoch = len(X)//batch_size

In [ ]:
for epoch in range(epochs):
    for batch in range(steps_per_epoch):
        input_gen = np.random.normal(0, 1, size=(batch_size, noise_dim))
        fake_data = generator.predict(input_gen)
        
        real_data = X[np.random.randint(0, X.shape[0], size=batch_size)]
        real_data = real_data.reshape((batch_size, width, height, channel))
        
        input_disc = np.concatenate((real_data, fake_data))

        label_disc = np.zeros(2*batch_size)
        label_disc[:batch_size] = 0.9
        label_disc[batch_size:] = 0.1
        loss_disc = discriminator.train_on_batch(input_disc, label_disc)

        label_gen = np.ones(batch_size)
        loss_gen = gan.train_on_batch(input_gen, label_gen)

    print("epoch: ", epoch)
    print("discriminator loss: ", loss_disc)
    print("generator loss: ", loss_gen)
    print("-"*80)
    
    if (epoch+1) % 20 == 0:
        show_generated_fabric("Generated Fabric", epoch)
        filename = 'generator_model_%03d.h5' % (epoch+1)
        generator.save(filename)

In [ ]:
import matplotlib.pyplot as plt
def show_data(X, title=""):
    plt.figure(figsize=(11,11))
    
    i = 1
    for img in X:
        plt.subplot(10, 10, i)
        plt.imshow(img.reshape((height, width,channel)))
        plt.axis('off')
        i+=1
        if i>100: break

    plt.suptitle(title, fontsize = 25)
    plt.show()
    
show_data(X, title="Original Dataset")